## supply_chain analysis

In [ ]:
import pyodbc
import pandas as pd
import numpy as np

In [5]:
%store -r connection_string
connection = pyodbc.connect(connection_string)

In [12]:
customers_query = 'SELECT * FROM customers'
products_query = 'SELECT * FROM products'
categories_departments_query = 'SELECT * FROM categories_departments'
orders_query = 'SELECT * FROM orders'
orders_ratio_query = 'SELECT * FROM orders_ratio'
orders_demographic_query = 'SELECT * FROM orders_demographic'
shipping_query = 'SELECT * FROM shipping'

In [13]:
customers = pd.read_sql_query(customers_query, connection)
products = pd.read_sql_query(products_query, connection)
categories_departments = pd.read_sql_query(categories_departments_query, connection)
orders = pd.read_sql_query(orders_query, connection)
orders_ratio = pd.read_sql_query(orders_ratio_query, connection)
orders_demographic = pd.read_sql_query(orders_demographic_query, connection)
shipping = pd.read_sql_query(shipping_query, connection)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12256\2733123061.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql_query(customers_query, connection)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12256\2733123061.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql_query(products_query, connection)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12256\2733123061.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  categories_departments = p

In [14]:
customers

,customer_id,first_name,last_name,segment,country,state,city,street,zipcode
0,11036,Bobby,Smith,Consumer,Puerto Rico,PR,Caguas,4464 Rocky Horse Harbour,725
1,11595,Zachary,Jones,Consumer,EE. UU.,CA,Modesto,777 Blue Hills Court,95355
2,10139,Mary,Gould,Consumer,Puerto Rico,PR,Caguas,6398 Indian Brook Valley,725
3,7277,Justin,Moore,Corporate,EE. UU.,CO,Littleton,7002 Blue Place,80126
4,10893,Lisa,Smith,Corporate,EE. UU.,CA,Riverside,3763 Noble Prairie Heights,92503
...,...,...,...,...,...,...,...,...,...
20647,16382,Rowan,Robbins,Home Office,Puerto Rico,PR,Caguas,1898 Velvet Villas,725
20648,18492,Hedy,Rivas,Consumer,Puerto Rico,PR,Caguas,1173 Silver Isle,725
20649,14086,Nelle,Hyde,Home Office,EE. UU.,NJ,Princeton,2407 Thunder Mountain,8540
20650,15424,Callie,Rodriguez,Home Office,EE. UU.,CA,Folsom,1836 Cozy View Orchard,95630


In [15]:
products

,product_id,product_name,category_id,department_id,price,product_image
0,1351,Dell Laptop,64,10,1500.00,http://images.acmesports.sports/Dell+Laptop
1,1350,Children's heaters,63,4,357.10,http://images.acmesports.sports/Children+heaters
2,1004,Field & Stream Sportsman 16 Gun Fire Safe,45,7,399.98,http://images.acmesports.sports/Field+%26+Stre...
3,835,Bridgestone e6 Straight Distance NFL Carolina,37,6,31.99,http://images.acmesports.sports/Bridgestone+e6...
4,44,adidas Men's F10 Messi TRX FG Soccer Cleat,3,2,59.99,http://images.acmesports.sports/adidas+Men%27s...
...,...,...,...,...,...,...
113,226,Bowflex SelectTech 1090 Dumbbells,11,3,599.99,http://images.acmesports.sports/Bowflex+Select...
114,677,TaylorMade White Smoke IN-12 Putter,31,6,99.99,http://images.acmesports.sports/TaylorMade+Whi...
115,777,Bag Boy M330 Push Cart,35,6,79.99,http://images.acmesports.sports/Bag+Boy+M330+P...
116,303,Garmin Forerunner 910XT GPS Watch,38,6,399.99,http://images.acmesports.sports/Garmin+Forerun...
